In [1]:
from torchcrf import CRF
import torch
import spacy
import torch.nn as nn
from torch_optimizer import SGDW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from metrics import f1score
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from multiprocessing import cpu_count
from platform import system
from os import environ
from sklearn.metrics import classification_report
from pandas import read_csv
from ast import literal_eval
import numpy as np
from itertools import chain
from utils import post_pad_sequences
from tensorflow.keras.utils import to_categorical

environ["TOKENIZERS_PARALLELISM"] = "false"
nlp = spacy.load("en_core_web_sm")
pl.seed_everything(seed=42)

Global seed set to 42


42

In [2]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
EPOCHS = 128
MAX_LEN = 128
WEIGHT_DECAY = 0
N_JOBS = cpu_count() if system() != "Windows" else 0

tag2idx = {'B': 0, 'I': 1, 'O': 2, 'E': 3, 'S': 4, '<': 5, ">":6, "$": 7}
pos2idx = {"NOUN": 0, "PROPN": 1, "VERB": 2, "ADJ": 3, "OTHER": 4, "<START>": 5, "<END>": 6, "<PAD>": 7}

In [3]:
class CRF4NER(pl.LightningModule):
    def __init__(self, 
                 num_tags=len(tag2idx),
                 train_dataset=None,
                 val_dataset=None,
                 test_dataset=None):

        super().__init__()
        self.crf = CRF(num_tags=num_tags,
                       batch_first=True)
        
        ## Hyperparameters ##
        self.batch_size = BATCH_SIZE
        self.learning_rate = LEARNING_RATE
        self.weight_decay = WEIGHT_DECAY
        ## Datasets ##
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset

    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size,
                          shuffle=True,
                          num_workers=N_JOBS,
                          drop_last=False)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)

    
    def predict_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)
        
        
    def forward(self, input_ids):
        pass


    def _shared_evaluation_step(self, batch, batch_idx):
        ids, masks, lbls = batch
        loss = -self.crf(ids, lbls, masks)
        pred = self.crf.decode(ids, masks)
        r, p, f1 = f1score(lbls.tolist(), pred)
        return loss, r, p, f1
    
        
    def training_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)

    
    def test_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        ids, masks, _ = batch 
        return self.crf.decode(ids, mask=masks)


    def configure_optimizers(self):
        optimizer = SGDW(self.parameters(), 
                         lr=self.learning_rate,
                         momentum=0.9,
                         nesterov=True,
                         weight_decay=self.weight_decay)

        return [optimizer]

In [4]:
data = read_csv("../data/train_290818.txt", 
                sep=" ",
                header=None, 
                encoding="utf-8").values.tolist()

text = [literal_eval(words) for (words, _, _) in data]
text = [[token.pos_ for token in nlp(' '.join(s))] for s in text]
text = [[p if (p == "NOUN" or p == "PROPN" or p == "VERB" or p == "ADJ") else "OTHER" for p in sent] for sent in text]
text = post_pad_sequences(text, maxlen=MAX_LEN, return_masks=True)
encoded_input = np.array([to_categorical([pos2idx[p] for p in sent], num_classes=len(tag2idx)) for sent in text["seq"]])

labels = [[l.split('-')[0] for l in literal_eval(labels)] for (_, labels, _) in data]
labels = post_pad_sequences(labels, maxlen=MAX_LEN, return_masks=False, start='<', end='>', pad='$')
labels = [[tag2idx[l] for l in lbls] for lbls in labels["seq"]]

L = len(labels)

dataset = TensorDataset(torch.LongTensor(encoded_input),
                        torch.BoolTensor(text["mask"]),
                        torch.LongTensor(labels))

train_sz, val_sz = L-int(0.1*L), int(0.1*L)
train_dataset, val_dataset = random_split(dataset, (train_sz, val_sz)) 

In [5]:
data = read_csv("../data/test_290818.txt", 
                sep=" ",
                header=None, 
                encoding="utf-8").values.tolist()

text = [literal_eval(words) for (words, _, _) in data]
text = [[token.pos_ for token in nlp(' '.join(s))] for s in text]
text = [[p if (p == "NOUN" or p == "PROPN" or p == "VERB" or p == "ADJ") else "OTHER" for p in sent] for sent in text]
text = post_pad_sequences(text, maxlen=MAX_LEN, return_masks=True)
encoded_input = np.array([to_categorical([pos2idx[p] for p in sent], num_classes=len(tag2idx)) for sent in text["seq"]])

labels = [[l.split('-')[0] for l in literal_eval(labels)] for (_, labels, _) in data]
labels = post_pad_sequences(labels, maxlen=MAX_LEN, start='<', end='>', pad='$', return_masks=False)
labels = [[tag2idx[l] for l in lbls] for lbls in labels["seq"]]

test_dataset = TensorDataset(torch.LongTensor(encoded_input),
                             torch.BoolTensor(text["mask"]),
                             torch.LongTensor(labels))                                                             

In [6]:
model = CRF4NER(train_dataset=train_dataset,
                val_dataset=val_dataset,
                test_dataset=test_dataset)

earlystopping_callback = EarlyStopping(monitor="val_f1score", 
                                       min_delta=1e-4, 
                                       patience=EPOCHS, 
                                       mode="max")

checkpoint_callback = ModelCheckpoint(dirpath="./",
                                      filename="crf-ner-val-f1score",
                                      save_top_k=1, 
                                      mode="max",
                                      monitor="val_f1score",
                                      save_weights_only=True)

trainer = pl.Trainer(accelerator="gpu",
                     max_epochs=EPOCHS,
                     precision=16,
                     log_every_n_steps=1,
                     callbacks=[earlystopping_callback, checkpoint_callback])

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(model)

Missing logger folder: /run/media/varun/New Volume/Suggestion-Mining-from-Noisy-Data/src_feat/lightning_logs
/home/varun/.local/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /run/media/varun/New Volume/Suggestion-Mining-from-Noisy-Data/src_feat exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | crf  | CRF  | 80    
------------------------------
80        Trainable params
0         Non-trainable params
80        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/varun/.local/lib/python3.10/site-packages/torch_optimizer/sgdw.py:112: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1174.)
  d_p = d_p.add(momentum, buf)


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [8]:
model.load_state_dict(torch.load(f"./crf-ner-val-f1score.ckpt")["state_dict"])
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_f1score        │            0.0            │
│         test_loss         │     745.1218872070312     │
│      test_precision       │            0.0            │
│        test_recall        │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 745.1218872070312,
  'test_recall': 0.0,
  'test_precision': 0.0,
  'test_f1score': 0.0}]

In [9]:
preds = list(chain.from_iterable(trainer.predict(model)))

preds = post_pad_sequences(preds,
                           start=tag2idx['<'], 
                           end=tag2idx['>'], 
                           pad=tag2idx['$'],
                           maxlen=len(labels[0]),
                           return_masks=False)["seq"]


preds = np.array(preds).flatten()                        
lbls = np.array(labels).flatten()

clf_rp = classification_report(lbls,
                               preds,
                               zero_division=0,
                               labels=[0, 1, 2, 3, 4, 5, 6, 7],
                               target_names=['B', 'I', 'O', 'E', 'S', 'START (<)', 'END (>)', 'PAD ($)'])

print("-"*80)
print(clf_rp)
print("-"*80)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 16it [00:00, ?it/s]

--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           B       0.00      0.00      0.00       311
           I       0.00      0.00      0.00        90
           O       0.82      0.93      0.87      7253
           E       0.00      0.00      0.00       309
           S       0.00      0.00      0.00       291
   START (<)       0.50      1.00      0.67       547
     END (>)       0.00      0.00      0.00       547
     PAD ($)       1.00      0.98      0.99     45352

    accuracy                           0.94     54700
   macro avg       0.29      0.36      0.32     54700
weighted avg       0.94      0.94      0.94     54700

--------------------------------------------------------------------------------
